In [106]:
import pandas as pd
import numpy as np
import random
from sklearn import linear_model

In [107]:
database=pd.read_pickle('311_clean.pkl')
database.head()
#database=pd.read_csv('311_time_difference')
#database.head()

,index,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Resolution Description,Borough,Status,difference
0,0,33022454,03/31/2016 02:10:00 AM,03/31/2016 02:30:31 AM,NYPD,New York City Police Department,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,11204,The Police Department responded to the complai...,BROOKLYN,Closed,20.516667
1,1,33023558,03/31/2016 01:48:58 AM,03/31/2016 03:02:20 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11367,The Police Department responded and upon arriv...,QUEENS,Closed,73.366667
2,2,33024431,03/31/2016 01:43:32 AM,03/31/2016 01:55:13 AM,NYPD,New York City Police Department,Noise - Commercial,Banging/Pounding,Club/Bar/Restaurant,10009,The Police Department responded to the complai...,MANHATTAN,Closed,11.683333
3,3,33024039,03/31/2016 01:42:59 AM,03/31/2016 02:45:19 AM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,11218,The Police Department issued a summons in resp...,BROOKLYN,Closed,62.333333
4,4,33024694,03/31/2016 01:37:16 AM,03/31/2016 02:10:11 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11365,The Police Department issued a summons in resp...,QUEENS,Closed,32.916667


In [108]:
#Since originally the data doesn't contain borough or time difference, I'm going to simmulate those attributes
#random.seed(42)
#database['difference']=np.random.choice(range(1, 1000), database.shape[0])
#database['borough']=np.random.choice(range(1,6), database.shape[0])

In [109]:
cols=[u'Agency',
     u'Complaint Type', u'Descriptor', u'Location Type',
       u'Incident Zip','difference']

In [110]:
database=database[cols]
database.head()

,Agency,Complaint Type,Descriptor,Location Type,Incident Zip,difference
0,NYPD,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,11204,20.516667
1,NYPD,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11367,73.366667
2,NYPD,Noise - Commercial,Banging/Pounding,Club/Bar/Restaurant,10009,11.683333
3,NYPD,Blocked Driveway,No Access,Street/Sidewalk,11218,62.333333
4,NYPD,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11365,32.916667


In [111]:
noise311_cut=database.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [112]:
x1 = noise311_cut.sample(10000)

In [113]:
#X = noise311_cut[['Agency','Complaint Type','Descriptor','Location Type','Incident Zip','borough']]
#X=noise311_cut[['Agency','Complaint Type','Descriptor','Location Type','borough']]
X=x1[['Agency','Complaint Type','Descriptor','Location Type','Incident Zip']]

In [114]:
Y=x1['difference'].as_matrix()
#Y=np.array(Y)
#Y=[item for sublist in Y for item in sublist]

In [115]:
x1 = noise311_cut.sample(10000)

In [116]:
X_dum= pd.get_dummies(X)

In [117]:
from sklearn.cross_validation import train_test_split

In [118]:
X_train,X_test,y_train,y_test = train_test_split(X_dum,Y,test_size=0.33,random_state=990) 

In [119]:
Lasso=linear_model.Lasso(fit_intercept=True,alpha=12) #try Ridge with a selected regularization parameter lambda

Lasso.fit(X_train,y_train)
# In the sample:
p_IS=Lasso.predict(X_train)
err_IS=p_IS-y_train
R_2_IS_Lasso=1-np.var(err_IS)/np.var(y_train)
print("The R-squared we found for IS Lasso is: {0}".format(R_2_IS_Lasso))

Lasso_coef=Lasso.coef_
############################################################################    
    
#Out of sample
p_OS=Lasso.predict(X_test)
err_OS=p_OS-y_test
R_2_OS_Lasso=1-np.var(err_OS)/np.var(y_test)
print("The R-squared we found for OS Lasso is: {0}".format(R_2_OS_Lasso))

The R-squared we found for IS Lasso is: 0.416279502508
The R-squared we found for OS Lasso is: 0.392463526912


##Lambda=10!! Future work: calibrate (although as long as we could see it, there's not too much difference

In [120]:
IS_rq=[]
OS_rq=[]
for i in range(1,51): 
    noise311_cut=database.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
    np.random.seed()
    x1 = noise311_cut.sample(10000,random_state=10*i)
    X=x1[['Agency','Complaint Type','Descriptor','Location Type','Incident Zip']]
    Y=x1['difference'].as_matrix()
    X_dum= pd.get_dummies(X)
    X_train,X_test,y_train,y_test = train_test_split(X_dum,Y,test_size=0.33,random_state=990) 
    Lasso=linear_model.Lasso(fit_intercept=True,alpha=10) #try Ridge with a selected regularization parameter lambda
    Lasso.fit(X_train,y_train)
    # In the sample:
    p_IS=Lasso.predict(X_train)
    err_IS=p_IS-y_train
    R_2_IS_Lasso=1-np.var(err_IS)/np.var(y_train)
    Lasso_coef=Lasso.coef_
    #Out of sample
    p_OS=Lasso.predict(X_test)
    err_OS=p_OS-y_test
    R_2_OS_Lasso=1-np.var(err_OS)/np.var(y_test)
    IS_rq.append(R_2_IS_Lasso)
    OS_rq.append(R_2_OS_Lasso)
print {'mean':np.mean(OS_rq),'min':np.min(OS_rq),'max':np.max(OS_rq),'std':np.std(OS_rq)}

{'std': 0.020861551818534607, 'max': 0.42507394146958621, 'min': 0.33991727725615195, 'mean': 0.38316111448008933}


WITH DIfferent lambdas:

In [105]:
results={}
for alp in range(6,40,2):
    IS_rq=[]
    OS_rq=[]
    results[alp]={}
    for i in range(1,51): 
        noise311_cut=database.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
        np.random.seed()
        x1 = noise311_cut.sample(10000,random_state=10*i)
        X=x1[['Agency','Complaint Type','Descriptor','Location Type','Incident Zip']]
        Y=x1['difference'].as_matrix()
        #Y=np.array(Y)
        #Y=[item for sublist in Y for item in sublist]
        X_dum= pd.get_dummies(X)
        X_train,X_test,y_train,y_test = train_test_split(X_dum,Y,test_size=0.33,random_state=990) 
        Lasso=linear_model.Lasso(fit_intercept=True,alpha=alp) #try Ridge with a selected regularization parameter lambda
        Lasso.fit(X_train,y_train)
        # In the sample:
        p_IS=Lasso.predict(X_train)
        err_IS=p_IS-y_train
        R_2_IS_Lasso=1-np.var(err_IS)/np.var(y_train)
        Lasso_coef=Lasso.coef_
        #Out of sample
        p_OS=Lasso.predict(X_test)
        err_OS=p_OS-y_test
        R_2_OS_Lasso=1-np.var(err_OS)/np.var(y_test)
        IS_rq.append(R_2_IS_Lasso)
        OS_rq.append(R_2_OS_Lasso)
    results[alp]={'mean':np.mean(OS_rq),'min':np.min(OS_rq),'max':np.max(OS_rq),'std':np.std(OS_rq)}
    print {'mean':np.mean(OS_rq),'min':np.min(OS_rq),'max':np.max(OS_rq),'std':np.std(OS_rq)}

KeyboardInterrupt: 